In [46]:
from datasets import load_dataset,DatasetDict,Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

In [47]:
# Load data set from huggingface
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

Repo card metadata block was not found. Setting CardData to empty.


In [48]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [49]:
df=[{'Name':i['Name'],'Symptoms':i['Symptoms']} for i in data_sample['train']]
df=pd.DataFrame(df)
df

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."
...,...,...
395,Urinary Stones (Kidney Stones),"Severe abdominal or back pain, blood in urine,..."
396,Osteoporosis,"Fragile bones, loss of height over time, back ..."
397,Rheumatoid Arthritis,"Joint pain, stiffness, swelling, fatigue, loss..."
398,Type 1 Diabetes,"Frequent urination, Increased thirst, Weight loss"


In [50]:
df['Symptoms'][0]

'Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness'

In [51]:

from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

In [52]:
# If you have an NVIDIA GPU attached, use 'cuda'
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    # If Apple Silicon, set to 'mps' - otherwise 'cpu' (not advised)
    try:
        device = torch.device('cpu')
    except Exception:
        device = torch.device('cpu')

In [53]:
device

device(type='cpu')

In [54]:

# The tokenizer turns texts to numbers (and vice-versa)
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

# The transformer
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

In [55]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [56]:
# Model params
BATCH_SIZE = 8

In [57]:
# Dataset Prep
class LanguageDataset(Dataset):
    """
    An extension of the Dataset object to:
      - Make training loop cleaner
      - Make ingestion easier from pandas df's
    """
    def __init__(self, df, tokenizer):
        self.labels = df.columns
        self.data = df.to_dict(orient='records')
        self.tokenizer = tokenizer
        x = self.fittest_max_length(df)  # Fix here
        self.max_length = x

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx][self.labels[0]]
        y = self.data[idx][self.labels[1]]
        text = f"{x} | {y}"
        tokens = self.tokenizer.encode_plus(text, return_tensors='pt', max_length=128, padding='max_length', truncation=True)
        return tokens

    def fittest_max_length(self, df):  # Fix here
        """
        Smallest power of two larger than the longest term in the data set.
        Important to set up max length to speed training time.
        """
        max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
        x = 2
        while x < max_length: x = x * 2
        return x

# Cast the Huggingface data set as a LanguageDataset we defined above
data_sample = LanguageDataset(df, tokenizer)


In [58]:
# Create train, valid
train_size = int(0.8 * len(data_sample))
valid_size = len(data_sample) - train_size
train_data, valid_data = random_split(data_sample, [train_size, valid_size])
     

In [59]:
# Make the iterators
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)

In [60]:
# Set the number of epochs
num_epochs = 10
     

# Training parameters
batch_size = BATCH_SIZE
model_name = 'distilgpt2'
gpu = 0
     

In [61]:
# Set the learning rate and loss function
## CrossEntropyLoss measures how close answers to the truth.
## More punishing for high confidence wrong answers
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token
     

# Init a results dataframe
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu',
                                'training_loss', 'validation_loss', 'epoch_duration_sec'])

In [62]:
# The training loop
for epoch in range(num_epochs):
    start_time = time.time()  # Start the timer for the epoch

    # Training
    ## This line tells the model we're in 'learning mode'
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training Loss': loss.item()})
        epoch_training_loss += loss.item()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    # Validation
    ## This line below tells the model to 'stop learning'
    model.eval()
    epoch_validation_loss = 0
    total_loss = 0
    valid_iterator = tqdm(valid_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            total_loss += loss
            valid_iterator.set_postfix({'Validation Loss': loss.item()})
            epoch_validation_loss += loss.item()

    avg_epoch_validation_loss = epoch_validation_loss / len(valid_loader)

    end_time = time.time()  # End the timer for the epoch
    epoch_duration_sec = end_time - start_time  # Calculate the duration in seconds

    new_row = {'transformer': model_name,
               'batch_size': batch_size,
               'gpu': gpu,
               'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss,
               'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}  # Add epoch_duration to the dataframe

    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(valid_loader)}")
     


















































































Training Epoch 1/10 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:56<00:00,  1.41s/it, Training Loss=0.716]





















Validation Epoch 1/10: 100%|██████████| 10/10 [00:04<00:00,  2.23it/s, Validation Loss=0.623]


Epoch: 1, Validation Loss: 0.6302040815353394



















































































Training Epoch 2/10 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:55<00:00,  1.40s/it, Training Loss=0.644]





















Validation Epoch 2/10: 100%|██████████| 10/10 [00:04<00:00,  2.22it/s, Validation Loss=0.612]


Epoch: 2, Validation Loss: 0.6099767684936523



















































































Training Epoch 3/10 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:57<00:00,  1.44s/it, Training Loss=0.503]





















Validation Epoch 3/10: 100%|██████████| 10/10 [00:04<00:00,  2.12it/s, Validation Loss=0.644]


...

Epoch: 8, Validation Loss: 0.8088713884353638



















































































Training Epoch 9/10 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:58<00:00,  1.47s/it, Training Loss=0.0977]





















Validation Epoch 9/10: 100%|██████████| 10/10 [00:04<00:00,  2.03it/s, Validation Loss=0.88]


Epoch: 9, Validation Loss: 0.8466261029243469



















































































Training Epoch 10/10 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:58<00:00,  1.47s/it, Training Loss=0.102]





















Validation Epoch 10/10: 100%|██████████| 10/10 [00:04<00:00,  2.08it/s, Validation Loss=0.889]

Epoch: 10, Validation Loss: 0.8548283576965332


In [73]:
input_str = "Kidney Failure"
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

output = model.generate(
    input_ids,
    max_length=20,
    num_return_sequences=1,
    do_sample=True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2
)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Kidney Failure
The problem with this is that the most common cause of failure in a given situation


In [74]:
def generate_output(input_str):
    input_ids = tokenizer.encode(input_str, return_tensors='pt')
    output = model.generate(
        input_ids,
        max_length=20,
        num_return_sequences=1,
        do_sample=True,
        top_k=8,
        top_p=0.95,
        temperature=0.5,
        repetition_penalty=1.2
    )
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded_output


In [75]:
import streamlit as st
st.title('Disease Prediction App')
input_str = st.text_input('Enter disease name:', '')
if st.button('Predict'):
        output = generate_output(input_str)
        st.write('Predicted Symptoms:', output)